# Voting alignment with China by topic

In [1]:
import plotly.express as px
data = px.data.iris()


from bokeh.plotting import figure, show, output_notebook
output_notebook(hide_banner=True)

import warnings
import pandas as pd
import numpy as np
import geopandas as gpd
from bokeh.models import (
    GeoJSONDataSource,
    Slider,
    CustomJS,
    LinearColorMapper,
    Range1d,
    ColumnDataSource,
    LabelSet,
    Span
)
from bokeh.models import HoverTool
from bokeh.layouts import column, row
import json
from bokeh.models import LinearAxis
from bokeh.palettes import PuBu, Greens, Reds
from IPython.display import display
from IPython.display import HTML

%pip install ipywidgets
%pip install colorcet
import colorcet
import ipywidgets as widgets


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Load world geometry data
world = gpd.read_file("data/ne_110m_admin_0_countries.shp")

AFR = ["Algeria","Angola","Benin","Botswana,Burkina Faso","Burundi","Cameroon","Congo","Côte d’Ivoire","Democratic Republic of the Congo","Djibouti","Egypt","Eritrea","Ethiopia","Gabon","The Gambia","Ghana","Kenya","Libya","Madagascar","Malawi","Mali","Mauritania","Mauritius","Morocco","Namibia","Nigeria","Rwanda","Senegal","Sierra Leone","South Africa","Somalia","Sudan","Togo","Tunisia","Uganda","Zambia",]
APAC = ["Afghanistan","Azerbaijan","Bahrain","Bangladesh","China","Fiji","India","Indonesia","Iraq","Japan","Jordan","Kazakhstan","Kuwait","Kyrgyzstan","Malaysia","Maldives","Marshall Islands","Mongolia","Nepal","Pakistan","Philippines","Qatar","Republic of Korea","Saudi Arabia","Sri Lanka","Thailand","United Arab Emirates","Uzbekistan","Viet Nam",]
GRULAC = ["Argentina","Bahamas","Bolivia (Plurinational State of)","Brazil","Chile","Costa Rica","Cuba","Ecuador","El Salvador","Guatemala","Honduras","Mexico","Nicaragua","Panama","Paraguay","Peru","Uruguay","Venezuela (Bolivarian Republic of)",]
WEOG = ["Australia","Austria","Belgium","Canada","Denmark","Finland","France","Germany","Iceland","Ireland","Italy","Luxembourg","Netherlands","Norway","Portugal","Spain","Switzerland","United Kingdom of Great Britain and Northern Ireland","United States of America",]
EG = ["Albania","Armenia","Bosnia and Herzegovina","Bulgaria","Czechia","Croatia","Estonia","Georgia","Hungary","Latvia","Lithuania","Montenegro","Poland","Republic of Moldova","Republic of North Macedonia","Romania","Russian Federation","Slovakia","Slovenia","Ukraine",]

# Function to assign color based on country group
def assign_color(country):
    if country in AFR:
        return "blue"
    elif country in APAC:
        return "green"
    elif country in GRULAC:
        return "red"
    elif country in WEOG:
        return "yellow"
    elif country in EG:
        return "orange"
    else:
        return "lightgray"


def get_country_group(country):
    if country in AFR:
        return "AFR"
    elif country in APAC:
        return "APAC"
    elif country in GRULAC:
        return "GRULAC"
    elif country in WEOG:
        return "WEOG"
    elif country in EG:
        return "EG"


def calculate_alignment(row):
    if row["Country"] == "China":
        return 1
    if row["China Vote"] == "Abstaining":
        return 0
    if row["Vote"] == row["China Vote"]:
        return 1
    else:
        return -1


def add_alignment_percentage(df):
    try:
        df["alignment_percentage"] = (
            df["alignment_score"] / df["Number of votes"]
        ) * 100
    except Exception:
        pass
    return df


def add_alignment(df):
    try:
        df["alignment"] = df["alignment_score"] / df["Number of votes"]
    except Exception:
        pass
    return df

def add_alignment_change(df):
    try:
        df["alignment_change"] = df["alignment"] - df["alignment"].shift(1)
    except Exception:
        pass
    return df

def add_fa_change(df):
    try:
        df["FT_ODI_change"] = df["FT_ODI"] - df["FT_ODI"].shift(1)
        df["AEI_FDI_change"] = df["AEI_FDI"] - df["AEI_FDI"].shift(1)
        df["BU_ODA_change"] = df["BU_ODA"] - df["BU_ODA"].shift(1)
        df["ODI_MOFCOM_change"] = df["ODI_MOFCOM"] - df["ODI_MOFCOM"].shift(1)
    except Exception:
        pass
    return df

def add_average_fa(df):
    try:
        df["Average_FA"] = (
            df[["FT_ODI", "AEI_FDI", "BU_ODA", "ODI_MOFCOM"]]
            .replace(0, np.nan)
            .mean(axis=1, skipna=True)
        )
        df["Average_FA"] = df["Average_FA"].fillna(0)
    except Exception:
        pass
    return df

def add_average_fa_change(df):
    try:
        df["Average_FA_change"] = df["Average_FA"] - df["Average_FA"].shift(1)
    except Exception:
        pass
    return df

def add_measures(df):
    df = add_alignment_percentage(df)
    df = add_alignment(df)
    df = add_average_fa(df)
    return df

def filter_country_group(row, country_group):
    return get_country_group(row["Country"]) == country_group


# Load and clean data:

# Load your data
oda = pd.read_csv("data/oda.csv")
votes = pd.read_csv("data/vote.csv")
df = pd.concat([oda, votes], ignore_index=True)
df_vac = pd.read_csv("data/vac.csv")


# Clean vaccine data
df_vac.fillna(0, inplace=True)
df_vac.rename(
    columns={
        "Destination": "Country",
        "Entry Date (date of delivery)": "Year",
        "Arrived (D)": "Quantity",
    },
    inplace=True,
)
df_vac = df_vac.drop(columns=["Region"])
df_vac["Quantity"] = df_vac["Quantity"].replace("-", np.nan)
df_vac.dropna(axis=0, subset=["Year", "Quantity"], inplace=True)
df_vac = df_vac[df_vac["Quantity"] != 0]
df_vac["Year"] = df_vac["Year"].replace("Aug", "01-Aug-21")
df_vac["Year"] = df_vac["Year"].replace("Oct-21", "01-Oct-21")
df_vac["Year"] = df_vac["Year"].replace("17 May 21", "17-May-21")
df_vac.head()
df_vac["Year"] = pd.to_datetime(df_vac["Year"], format="%d-%b-%y", dayfirst=True).dt.year
df_vac = df_vac.astype({"Year": "int", "Quantity": "float32"})
df_vac = df_vac.groupby(["Year", "Country"]).sum().reset_index()

# Clean vote data
df["Country Group"] = df["Country"].apply(get_country_group)
china_votes = df[df["Country"] == "China"][
    ["Session number", "Text title", "Vote"]
].rename(columns={"Vote": "China Vote"})
df = df.merge(
    china_votes,
    on=["Session number", "Text title"],
    suffixes=("", "_china"),
    how="left",
)
# Create a new column 'mapped_vote' that maps three values to -1, 0, and 1
vote_mapping = {"Against": -1, "Abstaining": 0, "In Favour": 1}
df["mapped_vote"] = df["Vote"].map(vote_mapping)
df["mapped_china_vote"] = df["China Vote"].map(vote_mapping)

df["alignment_score"] = df.apply(calculate_alignment, axis=1)
df_votes = (
    df.groupby(["Year", "Country", "Country Group", "Topic"])
    .agg(
        {
            "alignment_score": "sum",
            "Text title": "count",
        }
    )
    .reset_index()
)
df_votes.rename({"Text title": "Number of votes"}, axis=1, inplace=True)

# Clean FA data
df_fa = (
    df.groupby(["Year", "Country"])
    .agg(
        {
            "ODI_MOFCOM": "first",
            "BU_ODA": "first",
            "AEI_FDI": "first",
            "FT_ODI": "first",
        }
    )
    .reset_index()
)
df_fa[["FT_ODI", "AEI_FDI", "BU_ODA", "ODI_MOFCOM"]] = df_fa[
    ["FT_ODI", "AEI_FDI", "BU_ODA", "ODI_MOFCOM"]
].fillna(0)


# Global widgets
def get_filter(name):
    match name:
        case "fa_type":
            return widgets.Dropdown(
                options=["FT_ODI", "AEI_FDI", "BU_ODA", "ODI_MOFCOM", "Average_FA"],value="FT_ODI",
                description="FA Type:",
            )
        case "country_group":
            return widgets.Dropdown(
                options=["All", "AFR", "APAC", "GRULAC", "WEOG", "EG"], description="Country Group:", value="All"
            )
        case "country":
            return widgets.Dropdown(options=df.sort_values("Country")["Country"].unique(), description="Country:", value=df["Country"].unique().tolist()[0])
        case "topic":
            return widgets.Dropdown(
                options=["All"]+ df["Topic"].unique().tolist() ,
                description="Topic:",value="All"
            )


optionlist = {
    'fa_type': ["FT_ODI", "AEI_FDI", "BU_ODA", "ODI_MOFCOM", "Average_FA"],
    "country_group": ["All", "AFR", "APAC", "GRULAC", "WEOG", "EG"],
    "country": df.sort_values("Country")["Country"].unique(),
    "topic": ["All"]+ df["Topic"].unique().tolist()
}

In [3]:
# Overall voting alignment with China per vote type/topic, over time for all countries

votes = df_votes

def colour_map(x):
    if x > 0:
        return "green"
    elif x < 0:
        return "red"
    else:
        return "black"


def plot(country_group, topic):
    plot = None
    group_by = {"Year", "Country Group", "Topic"}
    filtered_votes = votes

    if country_group == "All":
        group_by.remove("Country Group")
    else:
        filtered_votes = df_votes[df_votes["Country Group"]== country_group]
    if topic == "All":
        group_by.remove("Topic")
    else:
        filtered_votes = filtered_votes[filtered_votes["Topic"] == topic]

    filtered_votes = filtered_votes.groupby(list(set(filtered_votes.columns) & group_by)).agg(
        {
            "alignment_score": "sum",
            "Number of votes": "sum",
        }
    ).reset_index()
    filtered_votes = add_measures(filtered_votes)
    filtered_votes = filtered_votes.astype({"Year": str,})
    x = filtered_votes["Year"].to_numpy()
    source = ColumnDataSource(
        data=dict(
            x=x,
            top=filtered_votes["alignment_percentage"],
            labels=filtered_votes["alignment_percentage"].apply(lambda x: str(round(x,2))+'%' if x else x),
            y_offset = filtered_votes["alignment_percentage"].apply(lambda x: 25*np.sign(x)),
            color=filtered_votes["alignment_percentage"].apply(colour_map)
        )
    )
    y_range = Range1d(start=-125, end=125)
    plot = figure(x_range=x,y_range=y_range,height=600, width=900, title="Overall Alignment by Topic over time")

    plot.vbar(
        width=0.9,
        fill_color="color",
        source=source
    )
    labels = LabelSet(
        x="x", y="top", text="labels", x_offset=-15, y_offset="y_offset", source=source
    )
    plot.add_layout(labels)
    show(plot)

country_group = get_filter("country_group")
topic = get_filter("topic")
plot3 = widgets.interactive(plot, country_group=country_group, topic=topic)

from IPython.display import display, Markdown, Latex

# fa_type = get_filter("fa_type")
# country_group = get_filter("country_group")
# topic = get_filter("topic")
for b in optionlist['country_group']:
    for c in optionlist['topic']:
        display(Markdown('## %s vs %s' % (b,c)))
        plot(b,c)


## All vs All

## All vs Sri Lanka

## All vs Non-Repatriation of Funds

## All vs Cooperation

## All vs Islamic Republic of Iran

...

## EG vs New and emerging technologies

## EG vs Climate change

## EG vs Environment

## EG vs Climate Change

## EG vs Colonialism